# Age Effect:
## Goal: Analyze the impact of system age on the mass-mass accretion rate relation

In [ ]:
#import relevant libraries and functions

from numpy import *
import astropy.units as u
from astropy.constants import G, M_jup, R_jup, M_earth, R_earth, L_sun, M_sun, R_sun
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import extinction as ex
import scipy.stats as st
import matplotlib.cm as cm

import importlib, functions
importlib.reload(functions)
from functions import line_lum, accr_lum, acc_rate, get_rate

In [ ]:
#read in CSV
db = pd.read_csv('recalculations.csv')
#optional: remove N06 points. They congregate at high accretion rates
#and make the age effect bigger than it likely is
#db = db[db['Reference']!='Natta 2006']
tracers = ['Ha','PaB','BrG']

In [ ]:
#get accretion rates
mass_used = []
for i in db.index:  
    d = db.loc[i,'Plx Distance']
    if isnan(d):
        d = db.loc[i,'Object Distance']
    R = db.loc[i,'Object Radius']
    M = db.loc[i,'New Mass']
    if isnan(M):
        M = db.loc[i,'Object Mass, Original']
    mass_used.append(M)
    #ignore if missing M or R
    if (isnan(M) or isnan(R)):
        continue
    for t in tracers:
        lf = db.loc[i,t + ' Line Flux']
        if isnan(lf):
            continue   
        #extinction corrections when necessary
        if db.loc[i,'Reference']=='Natta 2006':
            lf = ex.remove(db.loc[i,'A_J'],lf) 
        if (db.loc[i,'Reference']=='Kalari 2015' or db.loc[i,'Reference']=='Zhou 2014' or
            db.loc[i,'Reference']=='Wagner 2018' or db.loc[i,'Reference']=='Petrus 2020'):
            lf = ex.remove(db.loc[i,'A_V'],lf)      
        #get the accretion rate
        db.at[i,t + ' Accr Rate'] = get_rate(lf, d, t, R, M)
db['Adopted Mass'] = mass_used
db['Avg Accr Rate'] = db[[t + ' Accr Rate' for t in tracers]].mean(axis=1)

In [ ]:
test_age_orig = db[['Source','Object Mass, Original', 'Old Accr Rate', 'System Age']].dropna()
test_age_orig = test_age_orig[test_age_orig['System Age']!=0]

test_age = db[['Source','Adopted Mass', 'Avg Accr Rate', 'System Age']].dropna()

test_age_all = db[['Source','Object Mass, Original', 'Old Accr Rate', 'Adopted Mass', 'Avg Accr Rate','System Age']].dropna()

In [ ]:
#lists of parameters: 
mass_orig = log10(test_age_orig['Object Mass, Original'].tolist())
rate_orig = log10(test_age_orig['Old Accr Rate'].tolist())
age_orig = log10([1000000*i for i in test_age_orig['System Age'].tolist()])

mass = log10(test_age['Adopted Mass'].tolist())
rate = log10(test_age['Avg Accr Rate'].tolist())
age = log10([1000000*i for i in test_age['System Age'].tolist()])

## 1. Reproduce Figures 5.6 and 5.7 in Peck's Thesis: 
### 5.6: accretion rate as a function of age with points colored by mass
### 5.7: accretion rate as a function of mass with points colored by age

In [ ]:
#colormaps:
norm_mass_orig = [(i-min(mass_orig))/(max(mass_orig)-min(mass_orig)) for i in mass_orig]
norm_age_orig = [(i-min(age_orig))/(max(age_orig)-min(age_orig)) for i in age_orig]
massMap_orig = cm.viridis(norm_mass_orig)
ageMap_orig = cm.viridis(norm_age_orig)

norm_mass = [(i-min(mass))/(max(mass)-min(mass)) for i in mass]
norm_age = [(i-min(age))/(max(age)-min(age)) for i in age]
massMap = cm.viridis(norm_mass)
ageMap = cm.viridis(norm_age)

In [ ]:
f, (a1, a2) = plt.subplots(1,2,figsize=(12,4),dpi=400)

im1 = a1.scatter(age_orig,rate_orig,color=massMap_orig,s=120,alpha=0.6,edgecolors='none')
im2 = a2.scatter(age,rate,color=massMap,s=120,alpha=0.6,edgecolors='none')

plt.viridis()
c1 = f.colorbar(im1, ax=a1, label = 'log Mass ($M_{\odot}$)')
c2 = f.colorbar(im2, ax=a2, label = 'log Mass ($M_{\odot}$)')

im1.set_clim(min(mass_orig),max(mass_orig))
im2.set_clim(min(mass),max(mass))

a1.set_ylabel('$log \dot M$ ($M_{\odot}/yr$)')
a2.set_ylabel('$log \dot M$ ($M_{\odot}/yr$)')
a1.set_xlabel('log Age (Myr)')
a2.set_xlabel('log Age (Myr)')

a1.set_title('Original Data')
a2.set_title('Re-estimated Data')

In [ ]:
plt.figure(figsize=(7,7/1.625),dpi=400)

plt.scatter(mass,rate,color=ageMap,s=100,alpha=0.4,edgecolors='none')

plt.viridis()
plt.colorbar(label = 'log age (Myr)')
plt.clim(min(age),max(age))

## 2. Divide into age bins and plot each separately

Decide here whether you want to plot all of the original data, or only the original data points that were involved in our recalculations.

This matters: when you plot the recalculated data, you see a strong slope-steepening-with-age trend.

When you plot all of the original data, you do not see a slope-steepening-with-age trend. This might appear to show that the uniformly estimated data reveals a slope dependence on age.

However, when you only plot the original data that was then recalculated, you see the same steepening trend. This indicates this trend is, at least in part, due to selection bias and the samples included in our re-estimation.

In [ ]:
y1 = test_age_all[test_age_all['System Age']<=1]
y2 = test_age_all[(test_age_all['System Age']>1) & (test_age_all['System Age']<=3)]
y3 = test_age_all[test_age_all['System Age']>3]

#y1 = db[db['System Age']<=1]
#y2 = db[(db['System Age']>1) & (db['System Age']<=3)]
#y3 = db[db['System Age']>3]

In [ ]:
y1o = y1[['Object Mass, Original', 'Old Accr Rate']].dropna()
y2o = y2[['Object Mass, Original', 'Old Accr Rate']].dropna()
y3o = y3[['Object Mass, Original', 'Old Accr Rate']].dropna()

m1o, r1o = log10(y1o['Object Mass, Original'].tolist()),log10(y1o['Old Accr Rate'].tolist())
m2o, r2o = log10(y2o['Object Mass, Original'].tolist()),log10(y2o['Old Accr Rate'].tolist())
m3o, r3o = log10(y3o['Object Mass, Original'].tolist()),log10(y3o['Old Accr Rate'].tolist())

y1 = y1[['Adopted Mass', 'Avg Accr Rate']].dropna()
y2 = y2[['Adopted Mass', 'Avg Accr Rate']].dropna()
y3 = y3[['Adopted Mass', 'Avg Accr Rate']].dropna()

m1, r1 = log10(y1['Adopted Mass'].tolist()),log10(y1['Avg Accr Rate'].tolist())
m2, r2 = log10(y2['Adopted Mass'].tolist()),log10(y2['Avg Accr Rate'].tolist())
m3, r3 = log10(y3['Adopted Mass'].tolist()),log10(y3['Avg Accr Rate'].tolist())

In [ ]:
s1o,i1o = polyfit(m1o,r1o,1)
s2o,i2o = polyfit(m2o,r2o,1)
s3o,i3o = polyfit(m3o,r3o,1)

s1,i1 = polyfit(m1,r1,1)
s2,i2 = polyfit(m2,r2,1)
s3,i3 = polyfit(m3,r3,1)

R1o = st.pearsonr(m1o,r1o)[0]
R2o = st.pearsonr(m2o,r2o)[0]
R3o = st.pearsonr(m3o,r3o)[0]

R1 = st.pearsonr(m1,r1)[0]
R2 = st.pearsonr(m2,r2)[0]
R3 = st.pearsonr(m3,r3)[0]

In [ ]:
f = plt.figure(figsize=(10,7),dpi=750)

#set subplots grid: two rows, two plots on first row, one centered plot on the bottom row
grid = plt.GridSpec(ncols=4,nrows=2,figure=f,wspace=0.4,hspace=0.3)
ax1 = f.add_subplot(grid[0,0:2])
ax2 = f.add_subplot(grid[0,2:],sharey=ax1)
ax3 = f.add_subplot(grid[1,1:3])

#scatterplot new masses and accretion rates
ax1.scatter(m1,r1,alpha=0.6,color='b',label='r='+str(round(R1,2)),edgecolors='none')
ax2.scatter(m2,r2,alpha=0.6,color='r',label='r='+str(round(R2,2)),edgecolors='none')
ax3.scatter(m3,r3,alpha=0.6,color='g',label='r='+str(round(R3,2)),edgecolors='none')

#scatterplot old masses and accretion rates in gray
ax1.scatter(m1o,r1o,alpha=0.15,color='k',label='r='+str(round(R1o,2)),edgecolors='none')
ax2.scatter(m2o,r2o,alpha=0.15,color='k',label='r='+str(round(R2o,2)),edgecolors='none')
ax3.scatter(m3o,r3o,alpha=0.15,color='k',label='r='+str(round(R3o,2)),edgecolors='none')

x=arange(-3,2,1)
#plot new best-fit lines
ax1.plot(x,s1*x+i1,color='b',label='y='+str(round(s1,2))+'x'+str(round(i1,2)))
ax2.plot(x,s2*x+i2,color='r',label='y='+str(round(s2,2))+'x'+str(round(i2,2)))
ax3.plot(x,s3*x+i3,color='g',label='y='+str(round(s3,2))+'x'+str(round(i3,2)))

#plot old best-fit lines
ax1.plot(x,s1o*x+i1o,alpha=0.15,color='k',label='y='+str(round(s1o,2))+'x'+str(round(i1o,2)))
ax2.plot(x,s2o*x+i2o,alpha=0.15,color='k',label='y='+str(round(s2o,2))+'x'+str(round(i2o,2)))
ax3.plot(x,s3o*x+i3o,alpha=0.15,color='k',label='y='+str(round(s3o,2))+'x'+str(round(i3o,2)))

ax1.set_xlim(-2.5,1); ax1.set_ylim(-14,-6)
ax2.set_xlim(-2.5,1); ax2.set_ylim(-14,-6)
ax3.set_xlim(-2.5,1); ax3.set_ylim(-14,-6)

ax1.set_title('$\leq$ 1 Myr')
ax2.set_title('1-3 Myr')
ax3.set_title('>3 Myr')

ax1.legend(frameon=False,fontsize=7)
ax2.legend(frameon=False,fontsize=7)
ax3.legend(frameon=False,fontsize=7)

f.text(0.04,0.5,'log $\dot M (M_{\odot}/yr)$',va='center',rotation='vertical',fontsize=12)
f.text(0.5,0.05,'log $M (M_{\odot})$',ha='center',fontsize=12)
f.suptitle('Mass vs Accretion Rate')